In [1]:
# Combining Datasets: merge and join 
# fast in-memory merge and join operations with pd.merge

import pandas as pd 
import numpy as np 

In [ ]:
# pd.merge will create subset wich is work 
# as manipulating relational data that forms foundation
# of most DB 

In [2]:
# one-to-one joins

df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
    'group': ['Accounting', 'Engineering',
    'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
    'hire_date': [2004, 2008, 2012, 2014]})

df1, df2 

(  employee        group
 0      Bob   Accounting
 1     Jake  Engineering
 2     Lisa  Engineering
 3      Sue           HR,
   employee  hire_date
 0     Lisa       2004
 1      Bob       2008
 2     Jake       2012
 3      Sue       2014)

In [3]:
# combine information into a single DataFrame
df3 = pd.merge(df1,df2)
df3 

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


In [4]:
# many-to-one joins

df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
    'supervisor': ['Carly', 'Guido', 'Steve']})

df3, df4

(  employee        group  hire_date
 0      Bob   Accounting       2008
 1     Jake  Engineering       2012
 2     Lisa  Engineering       2004
 3      Sue           HR       2014,
          group supervisor
 0   Accounting      Carly
 1  Engineering      Guido
 2           HR      Steve)

In [5]:
# как видим объединение происходит по одной из оси 
# как раз - здесь у нас это id ник 
pd.merge(df3,df4)

,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


In [7]:
# many-to-many joins

df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
'Engineering', 'Engineering', 'HR', 'HR'],
'skills': ['math', 'spreadsheets', 'software', 'math',
'spreadsheets', 'organization']})


In [8]:
# как видим в выборке получаем полное перемешивание 
# если бы мы были бы в sql нам бы для нормальной обработки потребовалась бы 
# агрегация вокруг какого то идентификатора - например имени работника 

pd.merge(df1,df5)

,employee,group,skills
0,Bob,Accounting,math
1,Bob,Accounting,spreadsheets
2,Jake,Engineering,software
3,Jake,Engineering,math
4,Lisa,Engineering,software
5,Lisa,Engineering,math
6,Sue,HR,spreadsheets
7,Sue,HR,organization


In [9]:
# Specification of the Merge Key
# "on" keyword

# для того чтобы объединить два дейтафрема - нам надо колонку по которой 
# они будут объединяться (совмещения то могут быть же разными)
# её и задаем через on=..
# колонка очевидно должна быть в двух таблицах (DataFrame-ах)

pd.merge(df1, df2, on="employee")

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


In [12]:
# the left_on / right_on keywords:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
'salary': [70000, 80000, 120000, 90000]})


In [13]:
df1,df3

(  employee        group
 0      Bob   Accounting
 1     Jake  Engineering
 2     Lisa  Engineering
 3      Sue           HR,
    name  salary
 0   Bob   70000
 1  Jake   80000
 2  Lisa  120000
 3   Sue   90000)

In [15]:
# задание сочетания для левой таблицы и для правой таблицы
# имен ячеек:
pd.merge(df1,df3,left_on="employee",right_on="name")

,employee,group,name,salary
0,Bob,Accounting,Bob,70000
1,Jake,Engineering,Jake,80000
2,Lisa,Engineering,Lisa,120000
3,Sue,HR,Sue,90000


In [16]:
# у нас получилась лишняя ячейка - её можно отбросить
# axis = 0 -> rows 
# axis = 1 -> columns 
pd.merge(df1, df3, left_on="employee", right_on="name").drop("name",axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


In [17]:
# the left_index / right_index keywords:
df1a = df1.set_index("employee")
df2a = df2.set_index("employee")
df1a, df2a 

(                group
 employee             
 Bob        Accounting
 Jake      Engineering
 Lisa      Engineering
 Sue                HR,
           hire_date
 employee           
 Lisa           2004
 Bob            2008
 Jake           2012
 Sue            2014)

In [18]:
pd.merge(df1a, df2a, left_index=True, right_index=True)

,group,hire_date
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


In [22]:
# аналогично без дополнительных слов
df1a.join( df2a )

,group,hire_date
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


In [23]:
# можно задать иное поведение отличное от стандартного при объединении
pd.merge(df1a, df3, left_index=True, right_on="name")

,group,name,salary
0,Accounting,Bob,70000
1,Engineering,Jake,80000
2,Engineering,Lisa,120000
3,HR,Sue,90000


In [24]:
# Specifying Set Arithmetic for Joins

df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
    'food': ['fish', 'beans', 'bread']},
    columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
    'drink': ['wine', 'beer']},
    columns=['name', 'drink'])

In [25]:
# у нас только одна строка из общих:
# по сути это inner join : pd.merge(df6,df7,how="inner")
pd.merge(df6,df7)

,name,food,drink
0,Mary,bread,wine


In [26]:
# чтобы увидеть все вхождения с дырками данных делаем outer join 
# по сути это left outer join по отношению к df6
pd.merge(df6, df7, how="outer")

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine
3,Joseph,NaN,beer


In [27]:
pd.merge(df7, df6, how="outer")

,name,drink,food
0,Mary,wine,bread
1,Joseph,beer,NaN
2,Peter,NaN,fish
3,Paul,NaN,beans


In [28]:
# по сути к левой таблице прибить правую 
pd.merge(df6,df7, how="left")

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine


In [29]:
# как видим если не включать outer join 
# поведение будет вполне себе логичным - основной таблицей 
# является левая - там где можно к ней добавляются данные из правой 
pd.merge(df7,df6, how="left")

,name,drink,food
0,Mary,wine,bread
1,Joseph,beer,NaN


In [30]:
# Overlapping Column Names: The suffixes Keyword 

df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
    'rank': [3, 1, 4, 2]})


In [31]:
# суффиксы будут добавлены автоматически при конфликтующих именах
pd.merge(df8,df9, on="name")

,name,rank_x,rank_y
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


In [32]:
# суффиксы можно задать вручную
pd.merge(df8, df9, on="name", suffixes=["_L","_R"])

,name,rank_L,rank_R
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2
